# Scrap data

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
def is_beer_entry(table_row):
    row_cells = table_row.findAll('td')
    beer_id = get_beer_id(row_cells[0].text)
    return ( len(row_cells) == 8 and beer_id )

def get_beer_id(cell):
    r = re.match(r'^(\d{1,4})\.$', cell)
    if r and len(r.groups()) == 1:
        return int(r.group(1))
    return None

In [7]:
def get_all_beers(html_soup):
    beer_list = []
    keys = ['id', 'name', 'brewery_name', 'brewery_location', 'style', 'size', 'abv', 'ibu']
    for table_row in html_soup.findAll('tr'):
        if is_beer_entry(table_row):
            row_values = [td.text for td in table_row.findAll('td')]
            row_values[0] = get_beer_id(row_values[0])
            beer_entry = dict(zip(keys, row_values))
            beer_list.append(beer_entry)
    return beer_list

In [8]:
html = urlopen("http://craftcans.com/db.php?search=all&sort=beerid&ord=desc&view=text")
html_soup = BeautifulSoup(html, 'html.parser')
beers_list = get_all_beers(html_soup)

In [9]:
df = pd.DataFrame(beers_list)
df.head(5)

,abv,brewery_location,brewery_name,ibu,id,name,size,style
0,4.5%,"Minneapolis, MN",NorthGate Brewing,50,2692,Get Together,16 oz.,American IPA
1,4.9%,"Minneapolis, MN",NorthGate Brewing,26,2691,Maggie's Leap,16 oz.,Milk / Sweet Stout
2,4.8%,"Minneapolis, MN",NorthGate Brewing,19,2690,Wall's End,16 oz.,English Brown Ale
3,6.0%,"Minneapolis, MN",NorthGate Brewing,38,2689,Pumpion,16 oz.,Pumpkin Ale
4,6.0%,"Minneapolis, MN",NorthGate Brewing,25,2688,Stronghold,16 oz.,American Porter


# Create DataFrame for breweries

In [10]:
breweries = df[['brewery_location', 'brewery_name']]
breweries = breweries.drop_duplicates().reset_index(drop=True)
breweries['id'] = breweries.index
breweries.head(5)

,brewery_location,brewery_name,id
0,"Minneapolis, MN",NorthGate Brewing,0
1,"Louisville, KY",Against the Grain Brewery,1
2,"Framingham, MA",Jack's Abby Craft Lagers,2
3,"San Diego, CA",Mike Hess Brewing Company,3
4,"San Francisco, CA",Fort Point Beer Company,4


# Create DataFrame for beers

Note: First we need to merge(df, breweries), then take the relevant columns only


In [11]:
beers = pd.merge(df,
                breweries,
                left_on=['brewery_name', 'brewery_location'],
                right_on=['brewery_name', 'brewery_location'],
                sort=True,
                suffixes=('_beer', '_brewery'))
beers.head(5)

,abv,brewery_location,brewery_name,ibu,id_beer,name,size,style,id_brewery
0,5.0%,"Bend, OR",10 Barrel Brewing Company,N/A,1436,Pub Beer,12 oz.,American Pale Lager,408
1,6.6%,"Gary, IN",18th Street Brewery,N/A,2265,Devil's Cup,12 oz.,American Pale Ale (APA),177
2,7.1%,"Gary, IN",18th Street Brewery,N/A,2264,Rise of the Phoenix,12 oz.,American IPA,177
3,9.0%,"Gary, IN",18th Street Brewery,N/A,2263,Sinister,12 oz.,American Double / Imperial IPA,177
4,7.5%,"Gary, IN",18th Street Brewery,N/A,2262,Sex and Candy,12 oz.,American IPA,177


In [12]:
beers = beers[['abv', 'ibu', 'id_beer', 'name', 'size', 'style', 'id_brewery']]
beers_columns_rename = {
    'id_beer': 'id',
    'id_brewery': 'brewery_id'
}
beers.rename(inplace=True, columns=beers_columns_rename)
beers.head(5)

,abv,ibu,id,name,size,style,brewery_id
0,5.0%,N/A,1436,Pub Beer,12 oz.,American Pale Lager,408
1,6.6%,N/A,2265,Devil's Cup,12 oz.,American Pale Ale (APA),177
2,7.1%,N/A,2264,Rise of the Phoenix,12 oz.,American IPA,177
3,9.0%,N/A,2263,Sinister,12 oz.,American Double / Imperial IPA,177
4,7.5%,N/A,2262,Sex and Candy,12 oz.,American IPA,177


# Tidy data

## City and State: 2 variables

In [13]:
breweries['city'] = breweries["brewery_location"].apply(lambda location: location.split(",")[0])
breweries["state"] = breweries["brewery_location"].apply(lambda location: location.split(",")[1])
breweries = breweries[["brewery_name", "city", "state", "id"]]
breweries.rename(inplace=True, columns={"brewery_name": "name"})
breweries.head(5)

,name,city,state,id
0,NorthGate Brewing,Minneapolis,MN,0
1,Against the Grain Brewery,Louisville,KY,1
2,Jack's Abby Craft Lagers,Framingham,MA,2
3,Mike Hess Brewing Company,San Diego,CA,3
4,Fort Point Beer Company,San Francisco,CA,4


## Strings to numerical values

In [14]:
def string_pct_to_float(value):
    stripped = str(value).strip('%')
    try:
        return float(stripped)/100
    except ValueError:
        return None

beers['abv'] = beers['abv'].apply(string_pct_to_float)

def string_to_int(value):
    try:
        return int(value)
    except ValueError:
        return None

beers['ibu'] = beers['ibu'].apply(string_to_int)

In [15]:
beers.head(5)

,abv,ibu,id,name,size,style,brewery_id
0,0.050,NaN,1436,Pub Beer,12 oz.,American Pale Lager,408
1,0.066,NaN,2265,Devil's Cup,12 oz.,American Pale Ale (APA),177
2,0.071,NaN,2264,Rise of the Phoenix,12 oz.,American IPA,177
3,0.090,NaN,2263,Sinister,12 oz.,American Double / Imperial IPA,177
4,0.075,NaN,2262,Sex and Candy,12 oz.,American IPA,177


## Size to Ounces

In [16]:
for possible_value in set(beers['size'].tolist()):
    print(possible_value)

24 oz.
32 oz.
16.9 oz.
16 oz. Alumi-Tek®	
12 OZ.
12 & 16 oz.
19.2 oz.
12 oz.
12 oz. Slimline
16 oz. Alumi-Tek®
12 oz. 
16 oz
8.4 oz.
19.2
12 ounce
16 oz.
24 oz. "Silo Can"
12 oz


In [18]:
def extract_ounces(value):
    stripped = value.strip('oz')
    match = re.match(r'(\d{1,2}\.*\d*)', value)
    if match:
        return float(match.group(0))
    return None

beers['ounces'] = beers['size'].apply(extract_ounces)
del beers['size']

# Save

In [19]:
beers.head(5)

,abv,ibu,id,name,style,brewery_id,ounces
0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0


In [21]:
breweries.head(5)

,name,city,state,id
0,NorthGate Brewing,Minneapolis,MN,0
1,Against the Grain Brewery,Louisville,KY,1
2,Jack's Abby Craft Lagers,Framingham,MA,2
3,Mike Hess Brewing Company,San Diego,CA,3
4,Fort Point Beer Company,San Francisco,CA,4


In [22]:
beers.to_csv("../datasets/beers.csv")
breweries.to_csv("../datasets/breweries.csv")